In [1]:
from ipywidgets import interact, IntSlider, IntRangeSlider, Play, Dropdown, fixed
import tifffile
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_interactions import ipyplot as iplt

from pathlib import Path
import numpy as np
import cv2

D:\miniconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\miniconda\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
D:\miniconda\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
files = Path("../../data/raw").glob("*.tif")
tiffs = tifffile.TiffSequence(files)
mov = tiffs.asarray()
mov.shape

(2250, 498, 467)

In [3]:
window = cv2.namedWindow('img', cv2.WINDOW_NORMAL)

In [12]:
cv2.imshow("img", (mov[0] // 30).astype(np.uint8))
cv2.waitKey(1)

-1

In [16]:
Play??

Init signature: Play(*args, **kwargs)
Source:        
class Play(_BoundedInt):
    """Play/repeat buttons to step through values automatically, and optionally loop.
    """
    interval = CInt(100, help="The maximum value for the play control.").tag(sync=True)
    step = CInt(1, help="Increment step").tag(sync=True)
    disabled = Bool(False, help="Enable or disable user changes").tag(sync=True)

    _view_name = Unicode('PlayView').tag(sync=True)
    _model_name = Unicode('PlayModel').tag(sync=True)

    _playing = Bool(help="Whether the control is currently playing.").tag(sync=True)
    _repeat = Bool(help="Whether the control will repeat in a continous loop.").tag(sync=True)
    show_repeat = Bool(True, help="Show the repeat toggle button in the widget.").tag(sync=True)
File:           d:\miniconda\lib\site-packages\ipywidgets\widgets\widget_int.py
Type:           MetaHasTraits
Subclasses:     


In [14]:
@interact(
    idx=Play(value=0, min=0, max=len(mov) - 1, step=1, interval=50), 
    clim=IntRangeSlider(value=[0, mov.max()], min=0, max=mov.max()),
    x=IntSlider(value=20, min=0, max=mov.shape[2] - 11),
    y=IntSlider(value=20, min=0, max=mov.shape[1] - 11),
)
def show_frame(idx, clim, x, y):
    cmin, cmax = clim
    # fig, (ax1, ax2) = plt.subplots(ncols=2)
    # ax1.imshow(mov[idx], vmin=cmin, vmax=cmax);
    width, height = 10, 10
    x = x - width // 2
    y = y - height // 2
    cv2.imshow("img", ((mov[idx] - cmin) / cmax * 255).astype(np.uint8))
    cv2.waitKey(1)
    # cv2.destroyWindow("img")

    # rect = patches.Rectangle((x, y), 10, 10, facecolor='none', linewidth=1, edgecolor='r')
    # ax1.add_patch(rect)
    # roi = mov[:, y - height // 2 : y + height // 2, x - width // 2 : x + width // 2]
    # ax2.plot(roi.mean(axis=1).mean(axis=1))
    # ax2.vlines([idx], *ax2.get_ylim(), color='red', linewidth=1, linestyle='dotted')

interactive(children=(Play(value=0, description='idx', interval=50, max=2249), IntRangeSlider(value=(0, 8192),…

## Projection

In [19]:
@interact(
    aggfun=[("None", None), ("mean", np.mean), ('std', np.std), ('max', np.max), ('p2p', np.ptp), ('min', np.min)],
    cmap=['viridis', 'jet', 'gray'],
    vlim=IntRangeSlider(value=(0, np.max(mov)), min=0, max=np.max(mov)),
)
def show_projection(aggfun, cmap, vlim):
    frame = aggfun(mov, axis=0) if not aggfun is None else mov[0]
    plt.imshow(frame, cmap=cmap, vmin=vlim[0], vmax=vlim[1])


interactive(children=(Dropdown(description='aggfun', options=(('None', None), ('mean', <function mean at 0x000…